Отчёт по заданию 2.2 Julia

1. Настройка среды
Установка Julia 1.11.7
from https://julialang.org

Для работы с проектом использовались следующие инструменты:

julia
using RDatasets        # Загрузка стандартных датасетов
using DataFrames       # Работа с табличными данными
using Random           # Генерация случайных чисел
using CategoricalArrays# Работа с категориальными переменными
using DecisionTree     # Алгоритмы деревьев решений

2. Загрузка данных
iris = dataset("datasets", "iris")
Датасет Iris содержит измерения 150 цветков ириса по 4 признакам:

Длина чашелистика (SepalLength)

Ширина чашелистика (SepalWidth)

Длина лепестка (PetalLength)

Ширина лепестка (PetalWidth)

Целевая переменная: Species (вид ириса) - 3 класса:

setosa

versicolor

virginica

3. Разделение данных на обучающую и тестовую выборки
70% — обучающая выборка (105 samples)
30% — тестовая выборка (45 samples)

julia
Random.seed!(1234)  # Фиксируем случайность для воспроизводимости

4. Построение модели дерева решений
julia
model = DecisionTreeClassifier(max_depth=4)
Параметр max_depth=4 ограничивает глубину дерева для предотвращения переобучения.

5. Пример прогноза для нового цветка
julia
new_flower = [5.1, 3.5, 1.4, 0.2]  # Характеристики нового цветка
Предсказанный вид: setosa

In [4]:
using RDatasets
using DataFrames
using Random
using CategoricalArrays
using DecisionTree

# Загружаем Iris dataset
iris = dataset("datasets", "iris")
println("Размер датасета: ", size(iris))
println("Классы: ", unique(iris.Species))

y = iris.Species
X = select(iris, Not(:Species))

# Преобразуем в категориальный тип
y_cat = categorical(y)

# Разделение на train/test (70/30)
Random.seed!(1234)
n = nrow(iris)
train_size = Int(round(0.7 * n))
indices = shuffle(1:n)
train_idx = indices[1:train_size]
test_idx  = indices[train_size+1:end]

X_train = Matrix(X[train_idx, :])
y_train = y_cat[train_idx]
X_test = Matrix(X[test_idx, :])
y_test = y_cat[test_idx]

println("\nРазмер обучающей выборки: ", size(X_train))
println("Размер тестовой выборки: ", size(X_test))

# Обучаем модель DecisionTree
model = DecisionTreeClassifier(max_depth=4)
DecisionTree.fit!(model, X_train, y_train)

# Предсказания
y_pred = DecisionTree.predict(model, X_test)

# Точность
accuracy = sum(y_pred .== y_test) / length(y_test)
println("\n" * "="^50)
println("РЕЗУЛЬТАТЫ КЛАССИФИКАЦИИ")
println("="^50)
println("Точность модели: ", round(accuracy*100, digits=2), "%")

# Матрица ошибок
println("\nМатрица ошибок:")
cm = DecisionTree.confusion_matrix(y_test, y_pred)
println(cm)

# Точность по классам
println("\nТочность по классам:")
classes = levels(y_cat)
for class in classes
    class_mask = y_test .== class
    if sum(class_mask) > 0
        class_accuracy = sum(y_pred[class_mask] .== y_test[class_mask]) / sum(class_mask)
        println("$class: ", round(class_accuracy*100, digits=2), "%")
    end
end

# Пример прогноза для нового цветка
println("\n" * "="^50)
println("ПРИМЕР ПРОГНОЗА")
println("="^50)

new_flower = [5.1, 3.5, 1.4, 0.2]'  # характеристики нового цветка
new_pred = DecisionTree.predict(model, new_flower)
println("Новый цветок: [5.1, 3.5, 1.4, 0.2]")
println("Предсказанный вид: ", new_pred[1])

Размер датасета: (150, 5)
Классы: ["setosa", "versicolor", "virginica"]

Размер обучающей выборки: (105, 4)
Размер тестовой выборки: (45, 4)

РЕЗУЛЬТАТЫ КЛАССИФИКАЦИИ
Точность модели: 93.33%

Матрица ошибок:
Classes:  ["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.9333333333333333
Kappa:    0.8978046934140803

Точность по классам:
setosa: 100.0%
versicolor: 100.0%
virginica: 85.71%

ПРИМЕР ПРОГНОЗА
Новый цветок: [5.1, 3.5, 1.4, 0.2]
Предсказанный вид: setosa


3×3 Matrix{Int64}:
 14   0   0
  0  10   0
  0   3  18

6. Выводы
Модель дерева решений успешно решает задачу классификации видов ириса

Наибольшее влияние на классификацию оказывают параметры лепестков

Класс setosa легко отделяется от остальных, между versicolor и virginica возможна путаница

Ограничение глубины дерева помогает избежать переобучения

Разделение на train/test позволяет оценить реальную обобщающую способность модели